<a href="https://colab.research.google.com/github/dvashtom/colab-simulation-project/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Create a number guessing game where the computer generates a random integer within a specified range (e.g., 1 to 100), and the user tries to guess it. The game should provide feedback (higher/lower) after each guess, count the number of attempts, and conclude when the user guesses correctly or runs out of attempts, displaying an appropriate end message.

## הגדרת מספר יעד אקראי

### Subtask:
הגרלת מספר שלם אקראי בטווח מסוים (לדוגמה, 1 עד 100) שעל המשתמש לנחש.


**Reasoning**:
To generate a random integer, I need to import the `random` module and then use its `randint` function to get a number between 1 and 100 (inclusive).



In [1]:
import random

target_number = random.randint(1, 100)
print(f"The target number has been generated. It's an integer between 1 and 100.")

The target number has been generated. It's an integer between 1 and 100.


## קבלת ניחוש מהמשתמש

### Subtask:
קבלת קלט מהמשתמש עבור הניחוש שלו, תוך בדיקה שהקלט הוא מספר שלם תקין.


### **Reasoning**:
To ensure the user's guess is valid, we need to implement a loop that continuously prompts for input until a valid integer within the range of 1 to 100 is provided. This involves using a `try-except` block to catch non-integer inputs and conditional statements to check if the number is within the correct range.

**Reasoning**:
Now I will write the Python code to implement the input validation loop, using a `try-except` block for integer conversion and conditional checks for range validation, storing the valid guess in a variable.



In [2]:
while True:
    try:
        user_guess = int(input("Guess the number (between 1 and 100): "))
        if 1 <= user_guess <= 100:
            break
        else:
            print("Your guess is out of range. Please enter a number between 1 and 100.")
    except ValueError:
        print("Invalid input. Please enter an integer.")

print(f"You guessed: {user_guess}")

KeyboardInterrupt: Interrupted by user